In [ ]:
import csv
import tracery
from tracery.modifiers import base_english
import pandas as pd
import numpy as np
import ipywidgets as ipw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

## Part 1 - Tracery Outfit Builder

Run the cells below to generate an outfit. You can also edit the Excel file (wardrobe.csv) to include items from your own wardrobe! Just make sure all the columns have the same total number of rows. If you don't have enough unique items to fill the cells, just repeat them.

NB: 'Light' and 'Dark' refers to the colour/shade of the garment. Fill the 'Light' columns with your bright yellows and baby pinks and the 'Dark' columns with your maroons and navys.

In [ ]:
#loads in the wardrobe Excel file.

with open('wardrobe.csv', 'rb') as m:
    wardrobe = np.genfromtxt(m, delimiter = ",", dtype = "unicode", skip_header=1)

In [ ]:
#Formats columns into strings suitable for tracery grammar.

light_top = [str(w) for w in wardrobe[:,0]]
light_top = ",".join(light_top)

dark_top = [str(w) for w in wardrobe[:,1]]
dark_top = ",".join(dark_top)

light_down = [str(w) for w in wardrobe[:,2]]
light_down = ",".join(light_down)

dark_down = [str(w) for w in wardrobe[:,3]]
dark_down = ",".join(dark_down)

shoes = [str(w) for w in wardrobe[:,4]]
shoes = ",".join(shoes)

light_layer = [str(w) for w in wardrobe[:,6]]
light_layer = ",".join(light_layer)

dark_layer = [str(w) for w in wardrobe[:,7]]
dark_layer = ",".join(dark_layer)

coat = [str(w) for w in wardrobe[:,8]]
coat = ",".join(coat)

accessory = [str(w) for w in wardrobe[:,9]]
accessory = ",".join(accessory)

In [ ]:
#Outfit builder.

rules = {
    'origin': '#[#setLight#][#setDark#][#setShoes#][#setOuter#]story#',
    'story' : ["Start with your #Ltopoption#. Pair it with a #Loption# colour like your #Ldownoption#. For shoes, wear your #shoes#. Layer with your #LLoption#."
               ,"Start with your #Dtopoption#. Pair it with a #Doption# colour like your #Ddownoption#. For shoes, wear your #shoes#. Layer with your #DLoption#."],
    'setLight': ["[Loption:lighter][Ldownoption:" + light_down + "][Ltopoption:" + light_top + "]"],
    'setDark': ["[Doption:darker][Ddownoption:" + dark_down + "][Dtopoption:" + dark_top + "]"],
    'setShoes' : ["[Soption:shoe][shoes:" + shoes + "]"],
    'setOuter' : ["[Ooption:layer][LLoption:" + light_layer + "][DLoption:" + dark_layer + "]"]
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

In [ ]:
#Run this as many times as you like to keep changing outfits until you find one you like.
outfit = (grammar.flatten("#origin#"))
print(outfit)

## Part 2 - Style Classifier

Now that you have an outfit, let's see what style it is. This model can categorise clothing into Formal/Casual/Party. Take this as advice as to whether your look is appropriate for the office, a night out or maybe just the grocery store.

In [ ]:
#Loads in the fashion dataset.
dataset = pd.read_csv("fashiondata.csv")

In [ ]:
#TF/IDF Vectoriser
vectoriser = TfidfVectorizer(lowercase=True, ngram_range=(2, 3))
tfidf_model = vectoriser.fit(dataset["text"])

In [ ]:
#Sanity check the scores for each label
vectorised_formal = tfidf_model.transform(dataset[dataset["label"] == 'Formal']["text"]).todense()
vectorised_casual = tfidf_model.transform(dataset[dataset["label"] == 'Casual']["text"]).todense()
vectorised_party = tfidf_model.transform(dataset[dataset["label"] == 'Party']["text"]).todense()
vocab = np.array(tfidf_model.get_feature_names())

formal_sums = pd.DataFrame(vectorised_formal.sum(axis = 0).T, index=vocab, columns = ["Formal"])
casual_sums = pd.DataFrame(vectorised_casual.sum(axis = 0).T, index=vocab, columns = ["Casual"])
party_sums = pd.DataFrame(vectorised_party.sum(axis = 0).T, index=vocab, columns = ["Party"])

#Print to check!
print(formal_sums["Formal"].sort_values(ascending=False).head(8), casual_sums["Casual"].sort_values(ascending=False).head(8), party_sums["Party"].sort_values(ascending=False).head(8))

In [ ]:
tfidf = tfidf_model.transform(dataset["text"]).todense()

In [ ]:
svd = TruncatedSVD(n_components = 12, n_iter = 100)
svd_topic_model = svd.fit(tfidf)
svd_topic_vectors = svd_topic_model.transform(tfidf)

In [ ]:
#Compare TF/IDF and SVD

features = [tfidf, svd_topic_vectors]
labels = np.array(dataset["label"], dtype = object)

for f in features:
    X_train, X_test, y_train, y_test = train_test_split(f, labels, test_size=0.3, random_state=0)
    gnb = GaussianNB()
    model = gnb.fit(X_train, y_train);
    y_pred = model.predict(X_test)
    num_incorrect = (y_test != y_pred).sum()
    total = y_test.shape[0]
    acc = (total - num_incorrect) / total * 100
    print("Number of mislabeled points out of a total %d points : %d, %0.3f" % (total, num_incorrect, acc))

In [ ]:
#Choosing TF/IDF to create model.
X_train, X_test, y_train, y_test = train_test_split(tfidf, labels, test_size=0.3, random_state=0)
gnb = GaussianNB()
model = gnb.fit(X_train, y_train);

In [ ]:
#Pass the generated outfit desciption through the model.

new_tfidf = tfidf_model.transform([outfit]).todense()
y_pred = model.predict(new_tfidf)

for i, t in enumerate([outfit]):
    print(t,"-> Style:", y_pred[i])

## Part 3 - Future Work

The limitations of tracery is that the machine doesn't actually learn the garment. This means it cannot take brand new inputs that have not been categorised prior. A random input might have a better chance of working with styling accessories rather than full outfits. Below is a simple example of this. Try typing in something like 'white floral midi dress'.

In [ ]:
dress = input("Enter dress : ")

rules2 = {
    'origin2': '#[#Accessory#][#Coat#]story#',
    'story' : ['Bought a new #input#? Accessorise with your #accessory#. You can either dress it up with your #heels# or down with your #shoes#. If it is cold, throw on your #coat#.'],
    'input' : [dress],
    'Accessory':["[Aoption:Accessory][accessory:" + accessory + "]"],
    'heels': ["black stilletos", "white platform heels", "mustard low heels"],
    'shoes' : ["white nike sneakers","black mary-janes","navy doc martens", "black converse"],
    'Coat' : ["[Coption:Coat][coat:" + coat + "]"]
}

grammar = tracery.Grammar(rules2)
grammar.add_modifiers(base_english)
new_add_in = (grammar.flatten("#origin2#"))
print(new_add_in)

In [ ]:
# A simple prototype for what this might look like as an app.

output1 = ipw.Textarea(placeholder="What should I wear today?", layout=ipw.Layout(width="650px"),  disabled=True)
output2 = ipw.Text(placeholder="What does this look like?", layout=ipw.Layout(width="400px"),  disabled=True)

def on_click(btn):
    if btn.description == "match":
        output1.value = outfit
    if btn.description == "style":
        output2.value = y_pred[i]

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width='120px', height='70px'))
    btn.style.button_color = 'lightblue'
    btn.on_click(on_click)
    return btn

row0 = ipw.HBox([mk_btn(d) for d in ("match", "style")])
ipw.VBox((output1, row0,output2))